In [14]:
import math
import random
import skimage.io as io
import matplotlib.pyplot as plt
import keras
import pickle
import numpy as np
# import cv2
import os
from os import listdir
from os.path import isfile, join
import sys
# import dlib
# import skvideo.io
import json
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import glob
import re
from collections import defaultdict

import nltk
from nltk.corpus import cmudict
import tensorflow as tf
import wer

In [15]:
!nvidia-smi

Mon May 13 08:26:19 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.48                 Driver Version: 410.48                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:3E:00.0 Off |                  N/A |
| 36%   59C    P2   204W / 250W |  10743MiB / 10989MiB |     16%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:3F:00.0 Off |                  N/A |
| 30%   51C    P2   104W / 250W |  10743MiB / 10989MiB |     18%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [16]:
os.environ['CUDA_VISIBLE_DEVICES']='2'

In [17]:
phoneme_list = [] 
phoneme_dict = {}

with open("/n/fs/scratch/jiaqis/cmudict-master/cmudict.phones", 'r') as fp:
    i = 0
    line = fp.readline()
    while line:
        phoneme = line.split()[0].strip()
        phoneme_property = line.split()[1].strip()
        phoneme_list.append((phoneme, phoneme_property))
        phoneme_dict[phoneme] = i+3
        line = fp.readline()
        i=i+1
        
phoneme_dict['START'] = 0
phoneme_dict["END"] = 1
phoneme_dict["UNK"] = 2
print(phoneme_list, phoneme_dict)

([('AA', 'vowel'), ('AE', 'vowel'), ('AH', 'vowel'), ('AO', 'vowel'), ('AW', 'vowel'), ('AY', 'vowel'), ('B', 'stop'), ('CH', 'affricate'), ('D', 'stop'), ('DH', 'fricative'), ('EH', 'vowel'), ('ER', 'vowel'), ('EY', 'vowel'), ('F', 'fricative'), ('G', 'stop'), ('HH', 'aspirate'), ('IH', 'vowel'), ('IY', 'vowel'), ('JH', 'affricate'), ('K', 'stop'), ('L', 'liquid'), ('M', 'nasal'), ('N', 'nasal'), ('NG', 'nasal'), ('OW', 'vowel'), ('OY', 'vowel'), ('P', 'stop'), ('R', 'liquid'), ('S', 'fricative'), ('SH', 'fricative'), ('T', 'stop'), ('TH', 'fricative'), ('UH', 'vowel'), ('UW', 'vowel'), ('V', 'fricative'), ('W', 'semivowel'), ('Y', 'semivowel'), ('Z', 'fricative'), ('ZH', 'fricative')], {'IY': 20, 'START': 0, 'W': 38, 'DH': 12, 'Y': 39, 'HH': 18, 'CH': 10, 'JH': 21, 'ZH': 41, 'END': 1, 'EH': 13, 'UNK': 2, 'NG': 26, 'TH': 34, 'AA': 3, 'B': 9, 'AE': 4, 'D': 11, 'G': 17, 'F': 16, 'AH': 5, 'K': 22, 'M': 24, 'L': 23, 'AO': 6, 'N': 25, 'IH': 19, 'S': 31, 'R': 30, 'EY': 15, 'T': 33, 'AW': 7,

In [18]:
pron_dict = cmudict.dict()

In [19]:
DATA_DIR = "/n/fs/scratch/jiaqis/LRS3-TED/"
SAVE_DIR = "/n/fs/scratch/jiaqis/LRS3-TED-Extracted/"

In [20]:
test_ID_list = json.load(open('test_ID_list.json', "r"))
trainval_ID_list = json.load(open('trainval_ID_list.json', "r"))

In [21]:
import logging

import numpy as np
import tensorflow as tf
from tensorflow.contrib import layers


GO_TOKEN = 0
END_TOKEN = 1
UNK_TOKEN = 2

In [22]:
def clean_pron(pron):
    """Remove stress from pronunciations."""
    return re.sub(r"\d", "", pron)

def make_triphones(pron):
    """Output triphones from a word's pronunciation."""
    if len(pron) < 3:
        return []
    # Junk on end is to make word boundaries work
    return ([((pron[idx - 2], pron[idx - 1]), pron[idx])
             for idx in range(2, len(pron))] + [(('#', '#'), pron[0])] +
            [((pron[-2], pron[-1]), '#')])
                                                
def triphone_probs(prons):
    """Calculate triphone probabilities for pronunciations."""
    context_counts = defaultdict(lambda: defaultdict(int))
    for pron in prons:
        for (context, phoneme) in make_triphones(pron):
            context_counts[context][phoneme] += 1
            
    for (context, outcomes) in context_counts.items():
        total_outcomes = sum(outcomes.values())
        for outcome, count in outcomes.items():
            context_counts[context][outcome] = float(count) / total_outcomes
        
    return context_counts

In [23]:
def prepare_data(filepath, img_size, keypoint_img_size, keypoint_size):
    # images
    # frames x rows x cols x channels
    visual_cube = []
    # keypoint features
    feature_cube = []
    features = json.load(open(filepath + ".json", 'r'))
    # Target Text/phonemes
    labels = []
    text = open(filepath+".txt", 'r').readline()
    words = text[5:].lower().strip().split()
    for word in words:
        word_phonemes = pron_dict[word][0]
        word_indices = [phoneme_dict[clean_pron(phon)] for phon in word_phonemes]
        labels.extend(word_indices)
        
    acc = 0
    for imgFilename in sorted(glob.glob(filepath + "_*_mouth.jpg")): # 
#         if 'mouth' in imgFilename: # 
#             continue               #
        x = image.img_to_array(
              image.load_img(imgFilename, target_size=img_size))/255.0
#         x = np.expand_dims(x, axis=0)
#         x = preprocess_input(x)
        visual_cube.append(x)
        
        mask = np.zeros((keypoint_img_size[0], keypoint_img_size[1], keypoint_size))
        framenum = str(int(imgFilename.split("_")[-2].split(".")[0])) # 
        f_feature = features[framenum]['mouthCoords']
        for ft_index in range(keypoint_size):
            # TODO: check range of outputs
            keypoint_x = min(223, f_feature[ft_index][0] - 1)
            keypoint_y = min(223, f_feature[ft_index][1] - 1)
            mask[keypoint_y, keypoint_x, ft_index] = 1.0
        feature_cube.append(mask)
        acc+=1
    return np.array(visual_cube), np.array(feature_cube), labels

In [24]:
def frame_wise_op(inputs, operation, **kwargs):
    # batch x timesteps x w x h x c
    # or
    # batch x timesteps x c
    inputs_flat = tf.reshape(inputs, tf.concat([tf.constant(-1, shape=(1,)), 
                                                tf.shape(inputs)[2:]], axis=0))
    print(inputs, inputs_flat)
    outputs_flat = operation(inputs_flat, **kwargs)

    output_shape = tf.concat([tf.shape(inputs)[:2], tf.shape(outputs_flat)[1:]], 0)
    outputs = tf.reshape(outputs_flat, output_shape)

    return outputs

In [68]:
def seq2seq(mode, features, labels, params):
    vocab_size = params['vocab_size']
    embed_dim = params['embed_dim']
    num_units = params['num_units']
    num_encoder_rnns = params['num_encoder_rnns']
    num_decoder_rnns = params['num_decoder_rnns']
    input_max_length = params['input_max_length']
    output_max_length = params['output_max_length']
    beam_width = params['beam_width']

    inp = features['input']
    output = features['output']
    batch_size = tf.shape(inp)[0]
    start_tokens = tf.zeros([batch_size], dtype=tf.int64)
    print(inp, start_tokens)
    train_output = tf.concat([tf.expand_dims(start_tokens, 1), output], 1)
    # b x f x w x h x c
    input_lengths = tf.reduce_sum(tf.to_int32(tf.reduce_sum(tf.to_int32(tf.not_equal(inp, 0.0)), 
                                                            axis=(2,3,4))>0), 1)
    # b x seq
    output_lengths = tf.reduce_sum(tf.to_int32(tf.not_equal(train_output, END_TOKEN)), 1)
    print(train_output, input_lengths, output_lengths, output)

    with tf.variable_scope('video_embed', reuse=tf.AUTO_REUSE):
        # 224 x 224 x 64
        conv1 = tf.layers.conv3d(
                         inp,
                         filters=64,
                         kernel_size=[3, 3, 3],
                         padding='same',
                         activation='relu'
                    )
        # 112 x 112 x 64
        down1 = frame_wise_op(conv1,
                            operation=tf.layers.max_pooling2d,
                            pool_size=[2, 2],
                            strides=2)
        # 112 x 112 x 128
        conv2 = tf.layers.conv3d(down1,
                     filters=128,
                     kernel_size=[3, 3, 3],
                     padding='same',
                     activation='relu')
        # 56 x 56 x 128
#         down2 = frame_wise_op(conv2,
#                             operation=tf.layers.max_pooling2d,
#                             pool_size=[2, 2],
#                             strides=2)
        down2 = tf.layers.max_pooling3d(conv2,
                            pool_size=[2, 2, 2],
                            strides=2)
        # 56 x 56 x 256
        conv3 = tf.layers.conv3d(down2,
                     filters=256,
                     kernel_size=[3, 3, 3],
                     padding='same',
                     activation='relu')
        # 28 x 28 x 256
        down3 = frame_wise_op(conv3,
                            operation=tf.layers.max_pooling2d,
                            pool_size=[2, 2],
                            strides=2)
        # 28 x 28 x 256
        conv4 =  tf.layers.conv3d(down3,
                     filters=256,
                     kernel_size=[3, 3, 3],
                     padding='same',
                     activation='relu')
        # 14 x 14 x 256
#         down4 = frame_wise_op(conv4,
#                             operation=tf.layers.max_pooling2d,
#                             pool_size=[2, 2],
#                             strides=2)
        down4 = tf.layers.max_pooling3d(conv4,
                            pool_size=[2, 2, 2],
                            strides=2)
        # 14 x 14 x 256
        conv5 =  tf.layers.conv3d(down4,
                     filters=256,
                     kernel_size=[3, 3, 3],
                     padding='same',
                     activation='relu')
        # 7 x 7 x 256
        down5 = frame_wise_op(conv5,
                            operation=tf.layers.max_pooling2d,
                            pool_size=[2, 2],
                            strides=2)
        # 256
        down5_shape = down5.shape.as_list()
        flattened = frame_wise_op(down5, 
                           operation=tf.reshape,
                           shape=(-1, down5_shape[-1]*down5_shape[-2]*down5_shape[-3]))
        fc6 = frame_wise_op(flattened, 
                           operation=tf.layers.dense,
                           units=256,
                           activation='relu')
    
    output_embed = layers.embed_sequence(
        train_output, vocab_size=vocab_size, embed_dim=embed_dim, scope='embed')
    with tf.variable_scope('embed', reuse=True):
        embeddings = tf.get_variable('embeddings')

    encoder_cells = [tf.contrib.rnn.GRUCell(num_units=num_units) for i in range(num_encoder_rnns)]
    stacked_encoder_cell = tf.nn.rnn_cell.MultiRNNCell(encoder_cells)
    
    encoder_outputs, encoder_final_state = tf.nn.dynamic_rnn(stacked_encoder_cell, fc6, dtype=tf.float32)

    tiled_context_vector = tf.tile(tf.reshape(encoder_final_state, (-1, 1, 256)),
                                       multiples=[1,tf.shape(output_embed)[1],1]
                                      )
    print(embeddings, tiled_context_vector)
    
#     def re_embed(x):
#         indices = tf.argmax(x, axis=-1)
#         return tf.nn.embedding_lookup(embeddings, indices)
    
    train_helper = tf.contrib.seq2seq.TrainingHelper(output_embed, output_lengths)
#     train_helper = tf.contrib.seq2seq.ScheduledOutputTrainingHelper(
#         inputs = output_embed,
#         sequence_length = output_lengths,
#         next_inputs_fn = re_embed,
#         auxiliary_inputs = tiled_context_vector, # condtional on inputs
#         sampling_probability = 0.0, # for fullly inference
#     )
#     train_helper = tf.contrib.seq2seq.ScheduledEmbeddingTrainingHelper(
#         output_embed, sequence_length=output_lengths, embedding=embeddings, 
#         sampling_probability=0.1
#     )
    
#     def re_embed_inference(outputs):
#         sample_ids = tf.argmax(x, axis=-1)
#         finished = tf.equal(sample_ids, END_TOKEN)
#         all_finished = tf.reduce_all(finished)
#         next_inputs = tf.cond(
#             all_finished,
#             # If we're finished, the next_inputs value doesn't matter
#             lambda: tf.nn.embedding_lookup(embeddings, start_tokens),
#             lambda: tf.nn.embedding_lookup(embeddings, sample_ids))
#         print("next_inputs:", next_inputs)
#         return tf.concat([tf.to_float32(next_inputs), encoder_final_state], axis=-1)
    
    pred_helper = tf.contrib.seq2seq.SampleEmbeddingHelper(
        embeddings,
        start_tokens=tf.to_int32(start_tokens), end_token=1,
    )
#     pred_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
#                         embeddings, start_tokens=tf.to_int32(start_tokens), end_token=1)
#     pred_helper =tf.contrib.seq2seq.InferenceHelper(
#                             sample_fn = (lambda outputs: outputs),
#                             sample_shape = tf.TensorShape([vocab_size]),
#                             sample_dtype = tf.float32,
#                             start_inputs = tf.nn.embedding_lookup(embeddings, start_tokens),
#                             end_fn = (lambda x: tf.argmax(x, axis=-1)==END_TOKEN),
#                             next_inputs_fn = re_embed_inference,
#                         )

    tiled_encoder_outputs = tf.contrib.seq2seq.tile_batch(
        encoder_outputs, multiplier=beam_width)
    tiled_encoder_final_state = tf.contrib.seq2seq.tile_batch(
        encoder_final_state, multiplier=beam_width)
    tiled_sequence_length = tf.contrib.seq2seq.tile_batch(
        input_lengths, multiplier=beam_width)
    
    def decode(helper, scope, reuse=None, beam=True):
        with tf.variable_scope(scope, reuse=reuse):
            if beam:
                attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                    num_units=num_units, memory=tiled_encoder_outputs,
                    memory_sequence_length=tiled_sequence_length)
            else:
                attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                    num_units=num_units, memory=encoder_outputs,
                    memory_sequence_length=input_lengths)

            decoder_cells = [tf.contrib.rnn.GRUCell(num_units=num_units) for i in range(num_decoder_rnns)]
            stacked_decoder_cell = tf.nn.rnn_cell.MultiRNNCell(decoder_cells)
            
#             cell = tf.contrib.rnn.GRUCell(num_units=num_units)

            attn_cell = tf.contrib.seq2seq.AttentionWrapper(
                stacked_decoder_cell, attention_mechanism, attention_layer_size=num_units / 2)
            out_cell = tf.contrib.rnn.OutputProjectionWrapper(
                attn_cell, vocab_size, reuse=reuse
            )
            if beam:
                decoder_initial_state = out_cell.zero_state(
                                            dtype=tf.float32, batch_size=batch_size * beam_width)
                decoder_initial_state = decoder_initial_state.clone(cell_state=tiled_encoder_final_state)
                decoder = tf.contrib.seq2seq.BeamSearchDecoder(
                            out_cell,
                            embeddings,
                            start_tokens = tf.to_int32(start_tokens),
                            end_token = 1,
                            initial_state = decoder_initial_state ,
                            beam_width = beam_width,
                            length_penalty_weight = 0.2,
#                             coverage_penalty_weight = 0.2
                        )
            else:
                decoder_initial_state = out_cell.zero_state(
                                            dtype=tf.float32, batch_size=batch_size)
                decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_final_state)
                decoder = tf.contrib.seq2seq.BasicDecoder(
                    cell=out_cell, helper=helper,
                    initial_state=decoder_initial_state)
            
            outputs = tf.contrib.seq2seq.dynamic_decode(
                decoder=decoder, output_time_major=False,
                impute_finished=(not beam), maximum_iterations=output_max_length
            )
            print(outputs)
            return outputs[0]
    train_outputs = decode(train_helper, 'decode', beam=False)
    
    pred_outputs = decode(pred_helper, 'decode', reuse=True, beam=True)    

    final_predicted_ids = pred_outputs.predicted_ids
    scores = pred_outputs.beam_search_decoder_output.scores
    print(final_predicted_ids, scores)

    tf.identity(train_outputs.sample_id[0], name='train_pred')
    weights = tf.to_float(tf.not_equal(train_output[:, :-1], 1))
    loss = tf.contrib.seq2seq.sequence_loss(
        train_outputs.rnn_output, output, weights=weights)
    train_op = layers.optimize_loss(
        loss, tf.train.get_global_step(),
        optimizer=params.get('optimizer', 'Adam'),
        learning_rate=params.get('learning_rate', 0.001),
        learning_rate_decay_fn = (lambda p1, p2: tf.train.exponential_decay(
                                                        p1,
                                                        p2,
                                                        1000,
                                                        0.9,
                                                        staircase=False,
                                                    )),
        summaries=['loss', 'learning_rate'])

    tf.identity(pred_outputs.predicted_ids[0, :, 0], name='predictions')
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=pred_outputs.predicted_ids[:, :, 0],
        loss=loss,
        train_op=train_op
    )

In [69]:
def tokenize_and_map(line, vocab):
    return [vocab.get(token, UNK_TOKEN) for token in line.split(' ')]

def make_input_fn(
        data_dir, subset, list_IDs, prons, phonemes,
        input_max_length, output_max_length,
        img_size, keypoint_img_size,
        keypoint_size,
        batch_size=1,
        input_process=tokenize_and_map, 
        output_process=tokenize_and_map):

    def input_fn():
        inp = tf.placeholder(tf.float32, shape=[None, None, img_size[0], img_size[1], img_size[2]], name='input')
        output = tf.placeholder(tf.int64, shape=[None, None], name='output')
        tf.identity(inp[0], 'input_0')
        tf.identity(output[0], 'output_0')
        return {
            'input': inp,
            'output': output,
        }, None

    def sampler():
        while True:
            rnd = random.randint(0, len(list_IDs)-1)
            v_ID = list_IDs[rnd]
            v_url, v_index = v_ID
            filepath = os.path.join(data_dir, subset, v_url, v_index)
            v_V, v_F, v_T = prepare_data(filepath, img_size, keypoint_img_size,
                                           keypoint_size)
            num_frames = v_V.shape[0]
            yield {
                    'input':  v_V[:min(num_frames,input_max_length), :, :, :], # v_F[:input_max_length]),
                    'output': v_T[:output_max_length - 1] + [END_TOKEN]
                }

    sample_me = sampler()

    def feed_fn():
        inputs, outputs = [], []
        input_length, output_length = 0, 0
        for i in range(batch_size):
            rec = sample_me.next()
            inputs.append(rec['input'])
            outputs.append(rec['output'])
            num_frames = inputs[-1].shape[0] #
#             print(inputs[-1].shape)
            input_length = max(input_length, num_frames)
            output_length = max(output_length, len(outputs[-1]))
        # Pad me right with </S> token.
        for i in range(batch_size):
            num_frames = inputs[i].shape[0]
            if input_length>num_frames:
                new_input_V = np.pad(inputs[i], [(input_length - num_frames, 0), (0,0), (0,0), (0,0)], 'constant')
                # new_input_F = np.pad(inputs[i][1], [output_length - num_frames, 0, 0, 0], 'constant')
                inputs[i] = new_input_V
            outputs[i] += [END_TOKEN] * (output_length - len(outputs[i]))
        return {
            'input:0': inputs,
            'output:0': outputs
        }

    return input_fn, feed_fn


def get_rev_vocab(vocab):
    return {idx: key for key, idx in vocab.iteritems()}

def get_formatter(keys, vocab):
    rev_vocab = get_rev_vocab(vocab)
    def to_str(sequence):
        tokens = [
            rev_vocab.get(x, "<UNK>") for x in sequence]
        return ' '.join(tokens)

    def format(values):
        res = []
        for key in keys:
            res.append("%s = %s" % (key, to_str(values[key])))
        return '\n'.join(res)
    return format

def get_editdist_formatter(keys, vocab):
    rev_vocab = get_rev_vocab(vocab)
    
    def to_str(sequence):
        tokens = [str(x) for x in sequence]
#             rev_vocab.get(x, "<UNK>") for x in sequence]
        return tokens

    def format(values):
        labels = values[keys[0]]  
        clean_labels = []
        for label in labels:
            if label == END_TOKEN:
                break
            else:
                clean_labels.append(label)
        clean_labels = to_str(clean_labels)
        
        preds =  values[keys[1]]
        clean_preds = []
        for pred in preds:
            if pred == END_TOKEN:
                break
            else:
                clean_preds.append(pred)
        clean_preds = to_str(clean_preds)
        wer.wer(clean_labels, clean_preds)
        return '\n====== WER '+ keys[1]+' ======\n'
    
    return format

In [70]:
def train_seq2seq(data_dir, subset, model_dir, list_IDs, prons, phonemes):
    params = {
        'vocab_size': 42,
        'batch_size': 8,
        'input_max_length': 100,
        'output_max_length': 100,
        'embed_dim': 100,
        'num_units': 256,
        'img_size': (120, 120, 3), 
        'keypoint_img_size': (224, 224),
        'keypoint_size': 20,
        'n_classes':39,
        'num_tokens': 42,
        'num_encoder_rnns': 2,
        'num_decoder_rnns': 2,
        'beam_width': 10
    }
    est = tf.estimator.Estimator(
        model_fn=seq2seq,
        model_dir=model_dir, params=params)

    input_fn, feed_fn = make_input_fn(
                            data_dir, subset, list_IDs, prons, phonemes,
                            params['input_max_length'], params['output_max_length'],
                            params['img_size'], params['keypoint_img_size'],
                            params['keypoint_size'],
                            params['batch_size'])

    # Make hooks to print examples of inputs/predictions.
    print_inputs = tf.train.LoggingTensorHook(
        ['output_0'], every_n_iter=100,
        formatter=get_formatter(['output_0'], phonemes))
    print_predictions = tf.train.LoggingTensorHook(
        ['predictions', 'train_pred'], every_n_iter=100,
        formatter=get_formatter(['predictions', 'train_pred'], phonemes))
    print_train_edit_distance = tf.train.LoggingTensorHook(
        ['output_0', 'train_pred'], every_n_iter=100,
        formatter=get_editdist_formatter(['output_0', 'train_pred'], phonemes))
    print_pred_edit_distance = tf.train.LoggingTensorHook(
        ['output_0', 'predictions'], every_n_iter=100,
        formatter=get_editdist_formatter(['output_0', 'predictions'], phonemes))

    est.train(
        input_fn=input_fn,
        hooks=[tf.train.FeedFnHook(feed_fn), print_inputs, print_predictions, 
                       print_train_edit_distance, print_pred_edit_distance],
        steps=100000)

In [ ]:
#     tf.logging.logger.setLevel(logging.INFO)
train_seq2seq(SAVE_DIR, 'trainval', './model/seq2seq_copiedstates_3D_2rnn', trainval_ID_list, pron_dict, phoneme_dict)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7a38ba1f50>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_device_fn': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': './model/seq2seq_copiedstates_3D_2rnn', '_train_distribute': None, '_save_summary_steps': 100}
INFO:tensorflow:Calling model_fn.
(<tf.Tensor 'input:0' shape=(?, ?, 120, 120, 3) dtype=float32>, <tf.Tensor 'zeros:0' shape=(?,) dtype=int64>)
(<tf.Tensor 'concat:0' shape=(?, ?) dtype=int64>, <tf.Tensor 'Sum_1:0' shape=(?,) dtype=int32>, <tf.Tensor 'Sum_2:0' shape=(?,) dtype=int32>, <tf.Tensor 'ou

INFO:tensorflow:loss = 0.8857359, step = 10616 (360.887 sec)
INFO:tensorflow:Saving checkpoints for 10619 into ./model/seq2seq_copiedstates_3D_2rnn/model.ckpt.
INFO:tensorflow:global_step/sec: 0.280974
INFO:tensorflow:output_0 = AH N D DH IH S IH Z W AH T SH IY S EH D END END END END END END
INFO:tensorflow:predictions = S OW DH AE T S W AH T AY D IH D END END END END END END END END END END END END END END END END END END
train_pred = W N D DH AE S IH Z W AH T SH IY S EH D END START START START START START
REF: 5  25 11 12 19 31 19 40 38 5 33 32 20 31 13 11
HYP: 38 25 11 12 4  31 19 40 38 5 33 32 20 31 13 11
EVA: S           S                                 
WER: 12.50%
INFO:tensorflow:
====== WER train_pred ======

REF: 5 25 11 12 19 31 19 40 38 5 33 32 20 31 13 11
HYP:   31 27 12    4  33 31 38 5 33    8  11 19 11
EVA: D S  S     D  S  S  S          D  S  S  S    
WER: 68.75%
INFO:tensorflow:
====== WER predictions ======

INFO:tensorflow:loss = 1.1812346, step = 10716 (355.916 sec

INFO:tensorflow:Saving checkpoints for 11424 into ./model/seq2seq_copiedstates_3D_2rnn/model.ckpt.
INFO:tensorflow:global_step/sec: 0.228061
INFO:tensorflow:output_0 = B AH T AY V R IY L AY Z D W IY D OW N T N IY D T UW END END END
INFO:tensorflow:predictions = DH AE T S W AH T AY W AA N T T UW T AO K AH B AW T T AH D EY END END END END END END END END END END END
train_pred = IH AH T DH W G IH L AH W D DH IH R OW N T N OW D W UW END START START
REF: 9  5 33 8  37 30 20 23 8 40 11 38 20 11 27 25 33 25 20 11 33 36
HYP: 19 5 33 12 38 17 19 23 5 38 11 12 19 30 27 25 33 25 27 11 38 36
EVA: S       S  S  S  S     S S     S  S  S              S     S    
WER: 54.55%
INFO:tensorflow:
====== WER train_pred ======

REF: 9             5 33 8 37 30 20 23 8  40 11 38 20 11 27 25 33 25 20 11 33 36
HYP: 12 4 33 31 38 5 33 8 38 3  25 33 33 36 33 6  22 5  9  7  33 33 5  11    15
EVA: S  I I  I  I         S  S  S  S  S  S  S  S  S  S  S  S     S  S     D  S 
WER: 95.45%
INFO:tensorflow:
====== WER pred

REF: 19 33 31 29 3 30 33 5 37 12 5 29 23 15 17 30 7 25 11
HYP: 8  33 31 5  3 30 33 5 37 12 5 31 23 15 17 30 7 25 11
EVA: S        S                    S                     
WER: 15.79%
INFO:tensorflow:
====== WER train_pred ======

REF: 19 33 31 29 3 30 33 5 37 12 5  29 23 15 17 30 7 25 11   
HYP:       8  38 3 25 33      33 36 33 6  22 5  9  7 33 19 33
EVA: D  D  S  S    S     D D  S  S  S  S  S  S  S    S  S  I 
WER: 89.47%
INFO:tensorflow:
====== WER predictions ======

INFO:tensorflow:loss = 0.8403414, step = 12316 (496.237 sec)
INFO:tensorflow:Saving checkpoints for 12322 into ./model/seq2seq_copiedstates_3D_2rnn/model.ckpt.
INFO:tensorflow:global_step/sec: 0.208713
INFO:tensorflow:output_0 = DH IH S M AE N AH F EH S T S IH N AW ER S P IY CH END END END END
INFO:tensorflow:predictions = AH N D AY W UH D L AY K T UW M EY K AH K EY S END END END END END END END END END END
train_pred = AY AE S IH EY T AH F AO S T S IH N AW ER S P IY D END START START START
REF: 12 19 31 24 4  25 5 

REF: 33 15 22 5  23 35 22
HYP: 8  15 22 19 23 35 22
EVA: S        S          
WER: 28.57%
INFO:tensorflow:
====== WER train_pred ======

REF: 33 15    22 5  23 35 22              
HYP: 34 4  26 22 39 36 37 13 30 20 24 5 10
EVA: S  S  I     S  S  S  S  I  I  I  I I 
WER: 171.43%
INFO:tensorflow:
====== WER predictions ======

INFO:tensorflow:loss = 0.90186256, step = 13116 (411.394 sec)
INFO:tensorflow:Saving checkpoints for 13173 into ./model/seq2seq_copiedstates_3D_2rnn/model.ckpt.
INFO:tensorflow:global_step/sec: 0.246429
INFO:tensorflow:output_0 = DH EH R W AA Z AO L S OW IH N IH K W AA L AH T IY END END END END END END END
INFO:tensorflow:predictions = DH AE T S AH G R EY T P L EY S T UW B IY END END END END END END END END END END END END END END END END END
train_pred = DH EH R Z AA Z B L S OW IH N IH K S AA L AH T IY END START START START START START START
REF: 12 13 30 38 3 40 6 23 31 27 19 25 19 22 38 3 23 5 33 20
HYP: 12 13 30 40 3 40 9 23 31 27 19 25 19 22 31 3 23 5 33 20
EV

...

INFO:tensorflow:predictions = Y UW K AE N T B IY S EY IH NG DH AE T AH B AW T M IY END END END END END END END END END END END END END END
train_pred = DH T S AH AA T AH OW N IH N M UW V IY Z END START START START START START START
REF: 19 33 31 25 3 33 32 27 25 19 25 24 36 37 20 40
HYP: 12 33 31 5  3 33 5  27 25 19 25 24 36 37 20 40
EVA: S        S       S                            
WER: 18.75%
INFO:tensorflow:
====== WER train_pred ======

REF: 19 33 31 25 3  33 32 27 25    19 25 24 36 37 20 40           
HYP: 39 36 22 4  25 33 9  20 31 15 19 26 12 4  33 5  9  7 33 24 20
EVA: S  S  S  S  S     S  S  S  I     S  S  S  S  S  S  I I  I  I 
WER: 118.75%
INFO:tensorflow:
====== WER predictions ======

INFO:tensorflow:loss = 0.67243886, step = 24216 (281.320 sec)
INFO:tensorflow:global_step/sec: 0.383721
INFO:tensorflow:output_0 = W IY M AY T P L EY AH V ER ZH AH N AH V AW ER S EH L V Z END
INFO:tensorflow:predictions = AH N D DH AE T S DH AH R IY Z AH N W AY Y UW W AA N T T UW K IY P IH 

INFO:tensorflow:loss = 0.6385289, step = 25016 (287.689 sec)
INFO:tensorflow:Saving checkpoints for 25019 into ./model/seq2seq_copiedstates_3D_2rnn/model.ckpt.
INFO:tensorflow:global_step/sec: 0.369267
INFO:tensorflow:output_0 = DH EY D IH V AY D AH D DH EH M AH P IH N T UW T UW G R UW P S END END
INFO:tensorflow:predictions = DH AE T S W AH T S HH AE P AH N IH NG N AW AH D EY Z END END END END END END END END END END END END END END
train_pred = DH AE W OW D AY D AH D DH EH M AH P IH N T UW T UW G R UW P S END START
REF: 12 15 11 19 37 8 11 5 11 12 13 24 5 29 19 25 33 36 33 36 17 30 36 29 31
HYP: 12 4  38 27 11 8 11 5 11 12 13 24 5 29 19 25 33 36 33 36 17 30 36 29 31
EVA:    S  S  S  S                                                          
WER: 16.00%
INFO:tensorflow:
====== WER train_pred ======

REF: 12 15 11 19 37 8  11 5 11 12 13 24 5 29 19 25 33 36 33 36 17 30 36 29 31
HYP: 12       4  33 31 38 5    33 31 18 4 29 5  25    19 26 25 7  5  11 15 40
EVA:    D  D  S  S  S  S    D  

INFO:tensorflow:global_step/sec: 0.388574
INFO:tensorflow:output_0 = N OW W AH N HH AE D EH V ER D AH N DH IH S END END END END END END END END END END
INFO:tensorflow:predictions = AH N D DH IH S IH Z AE N IH M P AO R T AH N T P OY N T END END END END END END END
train_pred = AY AW B AH N IH AE D EH V ER D AH N DH IH S END START START START START START START START START START
REF: 25 27 38 5 25 18 4 11 13 37 14 11 5 25 12 19 31
HYP: 8  7  9  5 25 19 4 11 13 37 14 11 5 25 12 19 31
EVA: S  S  S       S                                
WER: 23.53%
INFO:tensorflow:
====== WER train_pred ======

REF:   25 27 38 5  25 18    4 11 13 37 14 11       5 25 12 19 31      
HYP: 5 25 11 12 19 31 19 40 4 25 19 24 29 6  30 33 5 25 33 29 28 25 33
EVA: I    S  S  S  S  S  I    S  S  S  S  S  I  I       S  S  S  I  I 
WER: 111.76%
INFO:tensorflow:
====== WER predictions ======

INFO:tensorflow:loss = 0.6587065, step = 25916 (257.380 sec)
INFO:tensorflow:global_step/sec: 0.364787
INFO:tensorflow:output_0 

REF: 18 7 11 36 8  29 19 22 38 5 33 8 38 3     25 33
HYP: 12 4 33 31 38 5  33 31 18 4 29 5 25 19 26 25 7 
EVA: S  S S  S  S  S  S  S  S  S S  S S  S  I     S 
WER: 100.00%
INFO:tensorflow:
====== WER predictions ======

INFO:tensorflow:loss = 0.63935864, step = 26716 (291.007 sec)
INFO:tensorflow:Saving checkpoints for 26738 into ./model/seq2seq_copiedstates_3D_2rnn/model.ckpt.
INFO:tensorflow:global_step/sec: 0.215777
INFO:tensorflow:output_0 = IH N W AH T K W AA N T AH T IY END END END END END END END END END END END END END END END END END END
INFO:tensorflow:predictions = DH AE T S W AH T W IY W AA N T AH D END END END END END END END END END END END END END END END END END END END END
train_pred = AY T DH AH T K W AA N T AH T IY END START START START START START START START START START START START START START START START START START
REF: 19 25 38 5 33 22 38 3 25 33 5 33 20
HYP: 8  33 12 5 33 22 38 3 25 33 5 33 20
EVA: S  S  S                            
WER: 23.08%
INFO:tensorflow

REF: 27 23 11 14 29 13 30 5 25 33 31 39 5 26 17 14 10 19 23 11 30 5 25
HYP: 12 22 11 14 29 20 30 5 25 33 31 39 5 26 17 14 10 19 23 11 30 5 25
EVA: S  S           S                                                 
WER: 13.04%
INFO:tensorflow:
====== WER train_pred ======

REF: 27 23 11 14 29 13 30 5 25 33 31 39 5  26 17 14 10 19 23 11 30 5  25
HYP: 5  25 11          38 5 25    5  37 12 27 40 11 15 40 24 19 31 33 14
EVA: S  S     D  D  D  S       D  S  S  S  S  S  S  S  S  S  S  S  S  S 
WER: 86.96%
INFO:tensorflow:
====== WER predictions ======

INFO:tensorflow:loss = 0.52916104, step = 27616 (336.033 sec)
